In [1]:
#https://towardsdatascience.com/text-classification-with-pytorch-7111dae111a6

In [2]:
import torch
from torch.nn import functional as F
from torch.autograd import Variable

from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe
import spacy
spacy.prefer_gpu()
from wordcloud import WordCloud, STOPWORDS
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
import os

from sklearn.model_selection import train_test_split


train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
submit_sample_data = pd.read_csv('sample_submission.csv')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/dawidkubicki/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


## Plan of attack

1. Preprocess data
    a) Delete unused columns
    b) Clean data - as it done in tensorflow notebook
    c) Split data with train_test_split from scikit
    d) Prepare tokens with tf tokenizer or pytorch thing
    e) Pad sequence or Pytorch thing
    
2. Prepare the model
    a) simple LSTM
    b) Bidirectional LSTM
    c) BERT
    
3. Prepare function to get batches

4. Write training phase

5. Write evaluation

6. Submit it

In [4]:
#preprocessing the data
X_train = train_data.drop(columns=["id", "keyword", "location"])
X_test = test_data.drop(columns=["id", "keyword", "location"])

In [5]:
X_train.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    
    return emoji_pattern.sub(r'', text)

lemmatizer = WordNetLemmatizer()

X_train["text"] = X_train["text"].apply(lambda x: clean_text(x))
X_test["text"] = X_test["text"].apply(lambda x: clean_text(x))

X_train["text"] = X_train["text"].apply(lambda x: remove_emoji(x))
X_test["text"] = X_test["text"].apply(lambda x: remove_emoji(x))

X_train["text"] = X_train["text"].apply(lambda x: lemmatizer.lemmatize(x))
X_test["text"] = X_test["text"].apply(lambda x: lemmatizer.lemmatize(x))

In [9]:
df_train = X_train['text']
df_test = X_test['text']

punctuations = string.punctuation
nlp_spacy = spacy.load('en_core_web_sm')
stop_words = STOP_WORDS
parser = English()
stopwords = set(STOPWORDS)

def spacy_preprocessor(sentence):
    result = nlp_spacy(sentence)
    return result

df_train = df_train.apply(lambda x: spacy_preprocessor(x))
df_test = df_test.apply(lambda x: spacy_preprocessor(x))

In [14]:
X_train['text'] = df_train
X_test['text'] = df_test

In [17]:
X = X_train[[col for col in X_train.columns if not col == 'target']]
y = X_train['target']
X_Train, X_Valid, y_Train, y_Valid = train_test_split(X, y, test_size=0.2, random_state=20)

train_data = pd.concat([X_Train, y_Train], axis=1)
valid_data = pd.concat([X_Valid, y_Valid], axis=1)

# save them
!mkdir preprocessed_data
train_data.to_csv('preprocessed_data/train.csv', index=False)
valid_data.to_csv('preprocessed_data/valid.csv', index=False)
X_test.to_csv('preprocessed_data/test.csv', index=False)

mkdir: cannot create directory ‘preprocessed_data’: File exists
